<html>
<table width="100%" cellspacing="2" cellpadding="2" border="1">
<tbody>
<tr>
<td valign="center" align="center" width="25%"><img src="media/decartes.jpg"
alt="DeCART Icon" width="128" height="171"><br>
</td>
<td valign="center" align="center" width="75%">
<h1 align="center"><font size="+1">DeCART Summer School<br>
for<br>
Biomedical Data Science</font></h1></td>
<td valign="center" align="center" width="25%"><img
src="media/U_Health_stacked_png_red.png" alt="Utah Health
Logo" width="128" height="134"><br>
</td>
</tr>
</tbody>
</table>
<br>
</html>


# Working with Textual Data
#### &copy; Brian E. Chapman, Ph.D.


### To get the latest version of this lesson, execute the cell below

In [1]:
%matplotlib inline

In [2]:
import pymysql
import pandas as pd
import getpass
from textblob import TextBlob

In [17]:
conn = pymysql.connect(host="mysql",
                       port=3306,user="jovyan",
                       passwd=getpass.getpass("Enter MySQL passwd for jovyan"),db='mimic2')
cursor = conn.cursor()

Enter MySQL passwd for jovyan········


### Use Pandas and SQL to create a dataframe with the following:
* subject_id
* hospital admission id
* text of the radiology report
* Limit the number of reports to 10000

In [18]:
rad_data = \
pd.read_sql("""SELECT noteevents.subject_id, 
                      noteevents.hadm_id,
                      noteevents.text 
               FROM noteevents
               WHERE noteevents.category = 'RADIOLOGY_REPORT' LIMIT 10000""",conn)
rad_data.head(5)

,subject_id,hadm_id,text
0,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...
1,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...
2,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:43 AM\n ...
3,56,28766.0,\n\n\n DATE: [**2644-1-17**] 6:37 AM\n ...
4,56,28766.0,\n\n\n DATE: [**2644-1-19**] 12:09 PM\n ...


In [19]:
rad_data.shape

(10000, 3)

In [10]:
set(rad_data["category"])

{'RADIOLOGY_REPORT'}

## Section Splitting

<img src="http://medicine.utah.edu/dbmi/images/faculty/Chapman,Wendy_240x360.jpg"
alt="Wendy Chapman" width="128">

Clinical reports are often divided into sections. Physicians are taught to write their reports in the SOAP format: subjective information, objective information, assessment, and plan, and sections often reflect this process. Information each setion may be formatted differently. For example, subjective information is often narrative, telling a story of the patient's description of their past medical history and their current issues. The objective section may be much more telegraphic with shortened descriptions of measurements like blood pressure and heart rate, such as "120/80 88". You may need different NLP techniques to accurately extract information out of different sections. Sentence splitting, for example, may be different in subjective versus objective sections. Radiology reports are not in the SOAP format, but section identification can still be very important, because in these reports the radiologist describes what she sees on the radiographic image then sometimes summarizes her impression of what the observations may represent, such as "the infiltrate could be consistent with pneumonia." For some tasks, you may only be interested in the impression, which has more weight than the reasoning that occurs in the Findings section.

## Exercise

Write a function that returns the impression section of a report. The function will take two arguments:

1. The text to search for the impression section is the first positional argument.
1. A list of phrases to use to identify the impression section.

Not every report will have a section labeled "IMPRESSION". You can consider "INTERPRETATION" and "CONCLUSIONS" as synonyms for "IMPRESSION"

If you cannot identify the impresison section (or equivalent), return an empty string.

#### Hints

* Use a for loop  to iterate across
* Use the find method of a string to identify where in the string the impression section occurs
* Use slicing to grab the impression section


In [37]:
txt10 = rad_data.loc[10,"text"]
txt5 = rad_data.loc[5,"text"]

In [41]:

def find_impression(txt, search_terms=["IMPRESSION", "INTERPRETATION", "CONCLUSIONS"]):
    
    for term in search_terms:
        ind = txt.find(term)
        if ind != -1:
            return txt[ind:]
        return ""
        
find_impression(txt5)


'IMPRESSION: Stable cardiomegaly with pulmonary vascular redistribution and new\n     lingular opacity. This could either represent pneumonia vs. asymmetric\n     pulmonary edema.\n\n'

## Exercise 

Use ``apply()`` to create a new column in the ``rad_data`` named "IMPRESSIONS" with the string identified with ``find_impression()``.

In [42]:
rad_data ["IMPRESSIONS"] =\
     rad_data ["text"].apply(find_impression)
rad_data.head()

,subject_id,hadm_id,text,IMPRESSIONS
0,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...,
1,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...,
2,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:43 AM\n ...,IMPRESSION: Stable appearance of right pariet...
3,56,28766.0,\n\n\n DATE: [**2644-1-17**] 6:37 AM\n ...,IMPRESSION:\n \n Cardiomegaly and mild...
4,56,28766.0,\n\n\n DATE: [**2644-1-19**] 12:09 PM\n ...,IMPRESSION:\n \n Marked improvement in...


## How else might we need to break up a report?

* How would I break a report into sentences?
* How would I break a report into words?

In [32]:
import numpy as np
def do_math (func):
    return [func(i) for i in np.arange (-np.pi, np.pi, 0.3)]

## Group Exercise

### Create a single string with all the reports

#### Hints, etc.
* Use List Comprehension
* Use string joins
* Iterate over the rows of the data frame
* Use TextBlob to get words

In [33]:
do_math (np.sin)

[-1.2246467991473532e-16,
 -0.29552020666133955,
 -0.5646424733950351,
 -0.7833269096274831,
 -0.9320390859672262,
 -0.9974949866040543,
 -0.9738476308781954,
 -0.8632093666488744,
 -0.6754631805511518,
 -0.4273798802338313,
 -0.14112000805986885,
 0.15774569414324657,
 0.4425204432948506,
 0.6877661591839722,
 0.8715757724135869,
 0.9775301176650966,
 0.9961646088358409,
 0.9258146823277333,
 0.7727644875559894,
 0.5506855425976405,
 0.2794154981989292]

In [35]:
do_math(lambda x:x**2)

[9.869604401089358,
 8.074648808935484,
 6.459693216781608,
 5.024737624627733,
 3.769782032473857,
 2.6948264403199813,
 1.7998708481661054,
 1.0849152560122293,
 0.549959663858353,
 0.19500407170447642,
 0.020048479550599657,
 0.025092887396722666,
 0.21013729524284547,
 0.575181703088968,
 1.1202261109350904,
 1.8452705187812126,
 2.7503149266273343,
 3.8353593344734564,
 5.100403742319577,
 6.545448150165699,
 8.17049255801182]